In [1]:
import requests
import pandas as pd

# Definición de los parametros a emplear para extraer los datos.
params = {
    'dataset': 'daily-summaries',
    'startDate': '1957-06-30', # Periodo - Inicio
    'endDate': '2024-07-02', # Periodo - final
    'stations': 'AR000087418',  # Mendoza Aereo
    'format': 'json',
    'includeStationName': 'true',
}

# 
url = "https://www.ncei.noaa.gov/access/services/data/v1"

response=requests.get(url,params=params)

if response.status_code==200:
    info=response.json()
else:
    print(f"Estatus data 1:-> {response.status_code}")

data=pd.DataFrame(info)

data

,DATE,STATION,TAVG,NAME,TMIN,TMAX,PRCP,SNWD
0,1957-06-30,AR000087418,28,"MENDOZA AERO, AR",NaN,NaN,NaN,NaN
1,1957-07-01,AR000087418,35,"MENDOZA AERO, AR",NaN,NaN,NaN,NaN
2,1957-07-02,AR000087418,47,"MENDOZA AERO, AR",NaN,NaN,NaN,NaN
3,1957-07-03,AR000087418,61,"MENDOZA AERO, AR",NaN,NaN,NaN,NaN
4,1957-07-04,AR000087418,73,"MENDOZA AERO, AR",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
24137,2024-06-28,AR000087418,90,"MENDOZA AERO, AR",13,150,NaN,NaN
24138,2024-06-29,AR000087418,54,"MENDOZA AERO, AR",-20,NaN,NaN,NaN
24139,2024-06-30,AR000087418,68,"MENDOZA AERO, AR",NaN,NaN,NaN,NaN
24140,2024-07-01,AR000087418,99,"MENDOZA AERO, AR",-16,NaN,NaN,NaN


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24142 entries, 0 to 24141
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   DATE     24142 non-null  object
 1   STATION  24142 non-null  object
 2   TAVG     19786 non-null  object
 3   NAME     24142 non-null  object
 4   TMIN     22535 non-null  object
 5   TMAX     19113 non-null  object
 6   PRCP     9394 non-null   object
 7   SNWD     5 non-null      object
dtypes: object(8)
memory usage: 1.5+ MB


In [3]:
data['DATE'] = pd.to_datetime(data['DATE'])

numeric_columns = ['TAVG', 'TMIN', 'TMAX', 'PRCP', 'SNWD']
for i in numeric_columns:
    data[i] = data[i].map(lambda x: (float(x)/10))

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24142 entries, 0 to 24141
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   DATE     24142 non-null  datetime64[ns]
 1   STATION  24142 non-null  object        
 2   TAVG     19786 non-null  float64       
 3   NAME     24142 non-null  object        
 4   TMIN     22535 non-null  float64       
 5   TMAX     19113 non-null  float64       
 6   PRCP     9394 non-null   float64       
 7   SNWD     5 non-null      float64       
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 1.5+ MB


In [13]:
a = round(data['TAVG'].mean() - data['TMIN'].mean())
b = round(data['TMAX'].mean() - data['TAVG'].mean())
c = round(data['TMAX'].mean() - data['TMIN'].mean())

for index, row in data.iterrows():
    if not pd.isna(row['TAVG']) and pd.isna(row['TMIN']) and pd.isna(row['TMAX']):
        data.at[index, 'TMIN'] = row['TAVG'] - a
        data.at[index, 'TMAX'] = row['TAVG'] + b
    elif pd.isna(row['TAVG']) and not pd.isna(row['TMIN']) and pd.isna(row['TMAX']):
        data.at[index, 'TAVG'] = row['TMIN'] + a
        data.at[index, 'TMAX'] = row['TMIN'] + c
    elif pd.isna(row['TAVG']) and pd.isna(row['TMIN']) and not pd.isna(row['TMAX']):
        data.at[index, 'TAVG'] = row['TMAX'] - b
        data.at[index, 'TMIN'] = row['TMAX'] - c
    elif pd.isna(row['TAVG']) and not pd.isna(row['TMIN']) and not pd.isna(row['TMAX']):
        data.at[index, 'TAVG'] = (row['TMIN'] + row['TMAX']) / 2
    elif not pd.isna(row['TAVG']) and not pd.isna(row['TMIN']) and pd.isna(row['TMAX']):
        data.at[index, 'TMAX'] = 2 * row['TAVG'] - row['TMIN']
    elif not pd.isna(row['TAVG']) and pd.isna(row['TMIN']) and not pd.isna(row['TMAX']):
        data.at[index, 'TMIN'] = 2 * row['TAVG'] - row['TMAX']

data

,DATE,STATION,TAVG,NAME,TMIN,TMAX,PRCP,SNWD
0,1957-06-30,AR000087418,2.8,"MENDOZA AERO, AR",-4.2,9.8,NaN,NaN
1,1957-07-01,AR000087418,3.5,"MENDOZA AERO, AR",-3.5,10.5,NaN,NaN
2,1957-07-02,AR000087418,4.7,"MENDOZA AERO, AR",-2.3,11.7,NaN,NaN
3,1957-07-03,AR000087418,6.1,"MENDOZA AERO, AR",-0.9,13.1,NaN,NaN
4,1957-07-04,AR000087418,7.3,"MENDOZA AERO, AR",0.3,14.3,NaN,NaN
...,...,...,...,...,...,...,...,...
24137,2024-06-28,AR000087418,9.0,"MENDOZA AERO, AR",1.3,15.0,NaN,NaN
24138,2024-06-29,AR000087418,5.4,"MENDOZA AERO, AR",-2.0,12.8,NaN,NaN
24139,2024-06-30,AR000087418,6.8,"MENDOZA AERO, AR",-0.2,13.8,NaN,NaN
24140,2024-07-01,AR000087418,9.9,"MENDOZA AERO, AR",-1.6,21.4,NaN,NaN


In [20]:
cols = ['PRCP', 'SNWD']

for col in cols:
    data[col] = data[col].fillna(0)

data

,DATE,STATION,TAVG,NAME,TMIN,TMAX,PRCP,SNWD
0,1957-06-30,AR000087418,2.8,"MENDOZA AERO, AR",-4.2,9.8,0.0,0.0
1,1957-07-01,AR000087418,3.5,"MENDOZA AERO, AR",-3.5,10.5,0.0,0.0
2,1957-07-02,AR000087418,4.7,"MENDOZA AERO, AR",-2.3,11.7,0.0,0.0
3,1957-07-03,AR000087418,6.1,"MENDOZA AERO, AR",-0.9,13.1,0.0,0.0
4,1957-07-04,AR000087418,7.3,"MENDOZA AERO, AR",0.3,14.3,0.0,0.0
...,...,...,...,...,...,...,...,...
24137,2024-06-28,AR000087418,9.0,"MENDOZA AERO, AR",1.3,15.0,0.0,0.0
24138,2024-06-29,AR000087418,5.4,"MENDOZA AERO, AR",-2.0,12.8,0.0,0.0
24139,2024-06-30,AR000087418,6.8,"MENDOZA AERO, AR",-0.2,13.8,0.0,0.0
24140,2024-07-01,AR000087418,9.9,"MENDOZA AERO, AR",-1.6,21.4,0.0,0.0


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24142 entries, 0 to 24141
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   DATE     24142 non-null  datetime64[ns]
 1   STATION  24142 non-null  object        
 2   TAVG     24142 non-null  float64       
 3   NAME     24142 non-null  object        
 4   TMIN     24142 non-null  float64       
 5   TMAX     24142 non-null  float64       
 6   PRCP     24142 non-null  float64       
 7   SNWD     24142 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 1.5+ MB
